# Kinematics

In [20]:
import numpy as np

class Point:
    def set_position(self, x,y,z):
        self.position = dict(x=x, y=y, z=z)

In [24]:
O = Point()
O.set_position(1,2,3)

print(O.position)

{'x': 1, 'y': 2, 'z': 3}
